In [ ]:
#pip install pandas openpyxl geopandas matplotlib

In [4]:
import pandas as pd

# Charger les deux fichiers CSV
batiments = pd.read_csv("batiments.csv")
infra = pd.read_csv("infra.csv")

# Afficher les valeurs distinctes
print("Valeurs distinctes de type_batiment :")
print(batiments["type_batiment"].dropna().unique())

print("\nValeurs distinctes de type_infra :")
print(infra["type_infra"].dropna().unique())


Valeurs distinctes de type_batiment :
['habitation' 'hôpital' 'école']

Valeurs distinctes de type_infra :
['aerien' 'fourreau' 'semi-aerien']


In [3]:
# Load the shapefiles to inspect their column names
try:
    gdf_infrastructures_cols = gpd.read_file("infrastructures.shp")
    print("Columns in infrastructures.shp:")
    print(gdf_infrastructures_cols.columns)

    gdf_batiments_cols = gpd.read_file("batiments.shp")
    print("\nColumns in batiments.shp:")
    print(gdf_batiments_cols.columns)
except Exception as e:
    print(f"Error loading shapefiles: {e}")

Columns in infrastructures.shp:
Index(['infra_id', 'longueur', 'geometry'], dtype='object')

Columns in batiments.shp:
Index(['id_bat', 'nb_maisons', 'geometry'], dtype='object')


In [7]:
import unicodedata
import pandas as pd
import geopandas as gpd
from dataclasses import dataclass, field
from typing import List, Dict, Set, Optional
import math


# les paramètres métier

COST_PER_M = {
    "aerien": 500.0,
    "semi-aerien": 750.0,
    "semi-aérien": 750.0, 
    "fourreau": 900.0,
}
HOURS_PER_M = {
    "aerien": 2.0,
    "semi-aerien": 4.0,
    "semi-aérien": 4.0,
    "fourreau": 5.0,
}
WORKER_HOURLY = 300.0 / 8.0  
MAX_WORKERS_PER_INFRA = 4
HOSPITAL_AUTONOMY_H = 20.0
HOSPITAL_MARGIN = 0.20  # 20% marge
HOSPITAL_DEADLINE_H = HOSPITAL_AUTONOMY_H * (1.0 - HOSPITAL_MARGIN)  # 16h

# la répartition budgétaire (après phase 0 hospitalière)
PHASE_BUDGET_SPLITS = [0.40, 0.20, 0.20, 0.20]
BUDGET_TOL = 0.03  # 3% de tolérance de dépassement max


def _strip_accents_lower(s: Optional[str]) -> str:
    if s is None:
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn").lower()



# classes principales

@dataclass
class Infrastructure:
    id: str
    etat: str  # 'infra_intacte' ou 'a_remplacer'
    longueur: float
    type_infra: str = ""  # aerien ,semi-aerien','fourreau'
    batiments_desservis: Set[str] = field(default_factory=set)
    nb_maisons_total: int = 0

    # Calculs métier
    cout_mat: float = 0.0
    heures: float = 0.0               # worker-hours total
    cout_mo: float = 0.0
    cout_total: float = 0.0
    duree_eff: float = 0.0            # durée calendaire si 4 ouvriers (h)
    diff: float = 0.0                 # métrique principale
    est_reparee: bool = False
    phase_reparation: int = -1

    def precalc(self):
        t = self.type_infra
        cpm = COST_PER_M.get(t, None)
        hpm = HOURS_PER_M.get(t, None)
        if cpm is None or hpm is None:
            raise ValueError(f"type_infra inconnu pour l'infra {self.id}: '{t}'")

        self.cout_mat = self.longueur * cpm
        self.heures = self.longueur * hpm                 # somme des worker-hours
        self.cout_mo = self.heures * WORKER_HOURLY
        self.cout_total = self.cout_mat + self.cout_mo
        self.duree_eff = self.heures / MAX_WORKERS_PER_INFRA if MAX_WORKERS_PER_INFRA > 0 else float('inf')

        # diff = (cout_total * heures) / nb_prises_max_maison
        denom = self.nb_maisons_total if self.nb_maisons_total > 0 else None
        self.diff = (self.cout_total * self.heures) / denom if denom else float('inf')
        return self

    def est_operationnelle(self) -> bool:
        return self.etat == "infra_intacte" or self.est_reparee


@dataclass
class Batiment:
    id: str
    nb_maisons: int
    type_batiment: str = ""  #habitation ou hôpital ou école
    infrastructures: List[Infrastructure] = field(default_factory=list)
    difficulte_totale: float = 0.0
    phase: int = -1

    def calculer_difficulte(self):
        non_op = [i for i in self.infrastructures if not i.est_operationnelle()]
        self.difficulte_totale = sum(i.diff for i in non_op)
        return self.difficulte_totale

    def est_raccorde(self) -> bool:
        return all(i.est_operationnelle() for i in self.infrastructures)



# planificateur

class PlanificateurRaccordement:
    def __init__(self, xlsx_path, shp_infra_path, shp_bat_path,
                 batiments_csv="batiments.csv", infra_csv="infra.csv"):
        self.xlsx_path = xlsx_path
        self.shp_infra_path = shp_infra_path
        self.shp_bat_path = shp_bat_path
        self.batiments_csv = batiments_csv
        self.infra_csv = infra_csv

        self.df_reseau = None
        self.gdf_infrastructures = None
        self.gdf_batiments = None
        self.df_bats = None
        self.df_infras = None

        self.infrastructures: Dict[str, Infrastructure] = {}
        self.batiments: Dict[str, Batiment] = {}
        self.plan_raccordement: List[Dict] = []

    # charger les data
    def charger_donnees(self):
        print("Chargement des données...")
        #fichier 
        self.df_reseau = pd.read_excel(self.xlsx_path)

        # fichier des types
        self.df_bats = pd.read_csv(self.batiments_csv)
        self.df_infras = pd.read_csv(self.infra_csv)

        # les shapefiles 
        try:
            self.gdf_infrastructures = gpd.read_file(self.shp_infra_path)
            self.gdf_batiments = gpd.read_file(self.shp_bat_path)
        except Exception:
            # Pas bloquant pour la planification
            self.gdf_infrastructures = None
            self.gdf_batiments = None

        print(f"{len(self.df_reseau)} lignes de réseau chargées")

        # dict de type
        infra_id_col = "infra_id" if "infra_id" in self.df_infras.columns else ("id_infra" if "id_infra" in self.df_infras.columns else None)
        bat_id_col = "id_batiment" if "id_batiment" in self.df_bats.columns else ("batiment_id" if "batiment_id" in self.df_bats.columns else None)
        if infra_id_col is None or bat_id_col is None:
            raise ValueError("Colonnes d'identifiants introuvables dans les CSV (attendu: infra_id/id_infra et id_batiment/batiment_id)")

        infra_type_map = dict(zip(self.df_infras[infra_id_col].astype(str), self.df_infras["type_infra"].astype(str)))
        bat_type_map = dict(zip(self.df_bats[bat_id_col].astype(str), self.df_bats["type_batiment"].astype(str)))

        # créer les infrastructures
        for infra_id in self.df_reseau['infra_id'].astype(str).unique():
            ligne = self.df_reseau[self.df_reseau['infra_id'].astype(str) == infra_id].iloc[0]
            type_infra = infra_type_map.get(infra_id, str(ligne.get('type_infra', '')))
            type_infra = _strip_accents_lower(type_infra) 

            self.infrastructures[infra_id] = Infrastructure(
                id=infra_id,
                etat=str(ligne['infra_type']),
                longueur=float(ligne['longueur']),
                type_infra=type_infra
            )

        # créer les bâtiments & lier aux infras
        for bat_id in self.df_reseau['id_batiment'].astype(str).unique():
            data = self.df_reseau[self.df_reseau['id_batiment'].astype(str) == bat_id]
            nb = int(data['nb_maisons'].iloc[0])

            # type_batiment 
            type_bat = bat_type_map.get(bat_id, "")
            bat = Batiment(id=bat_id, nb_maisons=nb, type_batiment=type_bat)

            for _, row in data.iterrows():
                infra = self.infrastructures[str(row['infra_id'])]
                bat.infrastructures.append(infra)
                infra.batiments_desservis.add(bat_id)
                infra.nb_maisons_total += nb

            self.batiments[bat_id] = bat

        # rré-calculs infra (coûts, heures, diff)
        for infra in self.infrastructures.values():
            infra.precalc()

        # Difficultés bâtiment 
        for bat in self.batiments.values():
            bat.calculer_difficulte()

        print(f"{len(self.batiments)} bâtiments chargés")
        print(f"{len(self.infrastructures)} infrastructures chargées\n")

    #outil
    def _cout_restants(self) -> float:
        return sum(i.cout_total for i in self.infrastructures.values() if not i.est_operationnelle())

    def _infras_non_op_bat(self, bat: Batiment) -> List[Infrastructure]:
        # dédupliqué par id
        non_op = [i for i in bat.infrastructures if not i.est_operationnelle()]
        return list({i.id: i for i in non_op}.values())

    # planification
    def planifier_raccordement(self):
        print("Début de la planification...\n")

        cout_total_global = 0.0

        # phase 0 : HÔPITAL en priorité dure, contrôle temps (≤16h)
        hospital_candidates = [b for b in self.batiments.values()
                               if _strip_accents_lower(b.type_batiment) == "hopital" or _strip_accents_lower(b.type_batiment) == "hopital " or _strip_accents_lower(b.type_batiment) == "hôpital"]
        if hospital_candidates:
            hopital = hospital_candidates[0]  # s'il y en a plusieurs on prend le premier
            infras_h = self._infras_non_op_bat(hopital)
            if infras_h:
                # durée calendaire si tout en // avec ≤4 ouvriers/infra -> max des duree_eff
                duree_phase0 = max(i.duree_eff for i in infras_h) if infras_h else 0.0
                ok_hopital_temps = duree_phase0 <= HOSPITAL_DEADLINE_H

                #réparer
                cout_phase_mat = sum(i.cout_mat for i in infras_h)
                cout_phase_mo = sum(i.cout_mo for i in infras_h)
                cout_phase_total = cout_phase_mat + cout_phase_mo
                heures_phase = sum(i.heures for i in infras_h)
                for i in infras_h:
                    i.est_reparee = True
                    i.phase_reparation = 0

                hopital.phase = 0
                cout_total_global += cout_phase_total

                self.plan_raccordement.append({
                    "Phase": 0,
                    "Bâtiment": hopital.id,
                    "Type bâtiment": hopital.type_batiment,
                    "Maisons": hopital.nb_maisons,
                    "Difficulté": round(hopital.difficulte_totale, 2),
                    "Infras réparées": ", ".join(i.id for i in infras_h),
                    "Coût matériel (phase)": round(cout_phase_mat, 2),
                    "Coût MO (phase)": round(cout_phase_mo, 2),
                    "Coût total (phase)": round(cout_phase_total, 2),
                    "Coût cumulé": round(cout_total_global, 2),
                    "Heures (phase)": round(heures_phase, 2),
                    "Durée critique (h)": round(duree_phase0, 2),
                    "Hopital_OK_≤16h": "OK" if ok_hopital_temps else "ALERTE"
                })
                # recalcul difficultés (certaines infras partagées peuvent impacter d'autres bâtiments)
                for b in self.batiments.values():
                    b.calculer_difficulte()
            else:
                hopital.phase = 0
                self.plan_raccordement.append({
                    "Phase": 0,
                    "Bâtiment": hopital.id,
                    "Type bâtiment": hopital.type_batiment,
                    "Maisons": hopital.nb_maisons,
                    "Difficulté": 0.0,
                    "Infras réparées": "",
                    "Coût matériel (phase)": 0.0,
                    "Coût MO (phase)": 0.0,
                    "Coût total (phase)": 0.0,
                    "Coût cumulé": round(cout_total_global, 2),
                    "Heures (phase)": 0.0,
                    "Durée critique (h)": 0.0,
                    "Hopital_OK_≤16h": "OK"
                })

        # marquer comme phase 0 tout bâtiment déjà raccordé 
        for bat in self.batiments.values():
            if bat.phase == -1 and bat.est_raccorde():
                bat.phase = 0

        # estimer le coût restant et fixer les plafonds de phases 1..4
        remaining = self._cout_restants()
        phase_caps = [split * remaining for split in PHASE_BUDGET_SPLITS]

        # phases 1..4:budget par phase
        current_phase_num = 1
        batiments_restants = [b for b in self.batiments.values() if b.phase == -1]

        while current_phase_num <= 4 and batiments_restants:
            cap = phase_caps[current_phase_num - 1] if current_phase_num - 1 < len(phase_caps) else float('inf')
            tol = cap * BUDGET_TOL
            cout_phase_total = 0.0
            heures_phase_total = 0.0
            duree_phase_critique = 0.0

            placed_any = True
            while placed_any:
                # recalculer difficultés
                for b in batiments_restants:
                    b.calculer_difficulte()

                # trier 
                def key_b(b: Batiment):
                    infras_needed = self._infras_non_op_bat(b)
                    cout_needed = sum(i.cout_total for i in infras_needed)
                    priority = 0
                    tb = _strip_accents_lower(b.type_batiment)
                    if tb == "ecole" or tb == "école":
                        priority = -1  # passe devant si égalité parfaite
                    return (b.difficulte_totale, cout_needed, priority, -b.nb_maisons)

                batiments_restants.sort(key=key_b)

                placed_any = False
                for b in list(batiments_restants):
                    infras_needed = self._infras_non_op_bat(b)
                    if not infras_needed:
                        b.phase = current_phase_num
                        batiments_restants.remove(b)
                        placed_any = True
                        # rien à ajouter en coût
                        self.plan_raccordement.append({
                            "Phase": current_phase_num,
                            "Bâtiment": b.id,
                            "Type bâtiment": b.type_batiment,
                            "Maisons": b.nb_maisons,
                            "Difficulté": round(b.difficulte_totale, 2),
                            "Infras réparées": "",
                            "Coût matériel (phase)": 0.0,
                            "Coût MO (phase)": 0.0,
                            "Coût total (phase)": 0.0,
                            "Coût cumulé": round(cout_total_global, 2),
                            "Heures (phase)": 0.0,
                            "Durée critique (h)": round(duree_phase_critique, 2),
                        })
                        continue

                    cout_mat = sum(i.cout_mat for i in infras_needed)
                    cout_mo = sum(i.cout_mo for i in infras_needed)
                    cout_needed = cout_mat + cout_mo
                    # vérif plafond 
                    if cout_phase_total + cout_needed <= cap + tol or (cap == 0 and cout_phase_total == 0):
                        # réparer
                        for i in infras_needed:
                            i.est_reparee = True
                            i.phase_reparation = current_phase_num

                        # MAJ métriques phase/globale
                        cout_phase_total += cout_needed
                        heures_add = sum(i.heures for i in infras_needed)
                        heures_phase_total += heures_add
                        duree_crit_add = max((i.duree_eff for i in infras_needed), default=0.0)
                        duree_phase_critique = max(duree_phase_critique, duree_crit_add)
                        cout_total_global += cout_needed

                        b.phase = current_phase_num
                        batiments_restants.remove(b)
                        placed_any = True

                        self.plan_raccordement.append({
                            "Phase": current_phase_num,
                            "Bâtiment": b.id,
                            "Type bâtiment": b.type_batiment,
                            "Maisons": b.nb_maisons,
                            "Difficulté": round(b.difficulte_totale, 2),
                            "Infras réparées": ", ".join(i.id for i in infras_needed),
                            "Coût matériel (phase)": round(cout_mat, 2),
                            "Coût MO (phase)": round(cout_mo, 2),
                            "Coût total (phase)": round(cout_needed, 2),
                            "Coût cumulé": round(cout_total_global, 2),
                            "Heures (phase)": round(heures_add, 2),
                            "Durée critique (h)": round(duree_phase_critique, 2),
                        })
                        # sortir pour re-trier
                        break

                # si rien ne rentre dans le budget restant de la phase, on clôture
                if not placed_any:
                    break

            current_phase_num += 1

        # bâtiments non planifiés (si jamais)
        for b in batiments_restants:
            # on les pousse en dernière phase + 1 par sécurité
            last_phase = 5
            infras_needed = self._infras_non_op_bat(b)
            cout_mat = sum(i.cout_mat for i in infras_needed)
            cout_mo = sum(i.cout_mo for i in infras_needed)
            cout_needed = cout_mat + cout_mo
            for i in infras_needed:
                i.est_reparee = True
                i.phase_reparation = last_phase
            cout_total_global += cout_needed
            b.phase = last_phase
            self.plan_raccordement.append({
                "Phase": last_phase,
                "Bâtiment": b.id,
                "Type bâtiment": b.type_batiment,
                "Maisons": b.nb_maisons,
                "Difficulté": round(b.difficulte_totale, 2),
                "Infras réparées": ", ".join(i.id for i in infras_needed),
                "Coût matériel (phase)": round(cout_mat, 2),
                "Coût MO (phase)": round(cout_mo, 2),
                "Coût total (phase)": round(cout_needed, 2),
                "Coût cumulé": round(cout_total_global, 2),
                "Heures (phase)": round(sum(i.heures for i in infras_needed), 2),
                "Durée critique (h)": round(max((i.duree_eff for i in infras_needed), default=0.0), 2),
            })

        print("Planification terminée.")
        print(f"Coût total estimé : {round(cout_total_global, 2)} €\n")

    # pour afficher
    def afficher_plan(self) -> pd.DataFrame:
        df = pd.DataFrame(self.plan_raccordement)
        # les features choisit 
        preferred = ["Phase", "Bâtiment", "Type bâtiment", "Maisons", "Difficulté",
                     "Infras réparées", "Coût matériel (phase)", "Coût MO (phase)",
                     "Coût total (phase)", "Coût cumulé", "Heures (phase)", "Durée critique (h)"]
        cols = [c for c in preferred if c in df.columns] + [c for c in df.columns if c not in preferred]
        df = df[cols]
        print("Tableau du plan de raccordement :\n")
        print(df.to_string(index=False))
        return df


# exemple
if __name__ == "__main__":
    xlsx_path = "reseau_en_arbre.xlsx"
    shp_infra_path = "infrastructures.shp"
    shp_bat_path = "batiments.shp"

    planif = PlanificateurRaccordement(xlsx_path, shp_infra_path, shp_bat_path,
                                       batiments_csv="batiments.csv", infra_csv="infra.csv")
    planif.charger_donnees()
    planif.planifier_raccordement()
    df_plan = planif.afficher_plan()
    df_plan.to_excel("plan_raccordement_phases.xlsx", index=False)
    print("\nTableau enregistré sous 'plan_raccordement_phases.xlsx'")


Chargement des données...
6107 lignes de réseau chargées
381 bâtiments chargés
644 infrastructures chargées

Début de la planification...

Planification terminée.
Coût total estimé : 1457263.57 €

Tableau du plan de raccordement :

 Phase Bâtiment Type bâtiment  Maisons  Difficulté                                       Infras réparées  Coût matériel (phase)  Coût MO (phase)  Coût total (phase)  Coût cumulé  Heures (phase)  Durée critique (h) Hopital_OK_≤16h
     0  E000085       hôpital        1   211214.33                             P005500, P007990, P007447               18483.26          2921.87            21405.13     21405.13           77.92                9.35              OK
     1  E000194    habitation        1    20670.55                    P005100, P000732, P008001, P000719               18773.43          2816.02            21589.45     42994.58           75.09                5.39             NaN
     1  E000195    habitation        1        0.00                            